In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import FloatType
from pyspark import SparkContext
spark = SparkSession.builder.getOrCreate()


In [2]:
tagDF = spark.read.option("header", "true").option("delimiter", ",")\
    .option("inferSchema", "true").csv("../ml-latest-small/tags.csv")
tagDF = tagDF.select("movieID", "tag")
tagDF.show()

+-------+-----------------+
|movieID|              tag|
+-------+-----------------+
|  60756|            funny|
|  60756|  Highly quotable|
|  60756|     will ferrell|
|  89774|     Boxing story|
|  89774|              MMA|
|  89774|        Tom Hardy|
| 106782|            drugs|
| 106782|Leonardo DiCaprio|
| 106782|  Martin Scorsese|
|  48516|     way too long|
|    431|        Al Pacino|
|    431|         gangster|
|    431|            mafia|
|   1221|        Al Pacino|
|   1221|            Mafia|
|   5995|        holocaust|
|   5995|       true story|
|  44665|     twist ending|
|  52604|  Anthony Hopkins|
|  52604|  courtroom drama|
+-------+-----------------+
only showing top 20 rows



In [3]:
ratingDF = spark.read.option("header", "true").option("delimiter", ",")\
    .option("inferSchema", "true").csv("../ml-latest-small/ratings.csv")
ratingDF = ratingDF.select("movieID", "rating")
ratingDF.show()

+-------+------+
|movieID|rating|
+-------+------+
|      1|   4.0|
|      3|   4.0|
|      6|   4.0|
|     47|   5.0|
|     50|   5.0|
|     70|   3.0|
|    101|   5.0|
|    110|   4.0|
|    151|   5.0|
|    157|   5.0|
|    163|   5.0|
|    216|   5.0|
|    223|   3.0|
|    231|   5.0|
|    235|   4.0|
|    260|   5.0|
|    296|   3.0|
|    316|   3.0|
|    333|   5.0|
|    349|   4.0|
+-------+------+
only showing top 20 rows



In [4]:
# matching ratings with tags in here
joinedDF = ratingDF.join(tagDF,"movieID", "outer")
joinedDF = joinedDF.select("rating", "tag")
joinedDF.show(5)

+------+-----+
|rating|  tag|
+------+-----+
|   4.0|pixar|
|   4.0|pixar|
|   4.0|  fun|
|   4.0|pixar|
|   4.0|pixar|
+------+-----+
only showing top 5 rows



In [5]:
# taking the average rating for each tag in here
uniqueTagsDF = joinedDF.groupby("tag").sum("rating").select("tag", "sum(rating)")
uniqueTagsDF = joinedDF.groupby("tag").count()
uniqueTagsDF = joinedDF.groupby("tag").avg().sort("avg(rating)",ascending=False)

uniqueTagsDF.take(10)

[Row(tag='no dialogue', avg(rating)=5.0),
 Row(tag='creativity', avg(rating)=5.0),
 Row(tag='procedural', avg(rating)=5.0),
 Row(tag='human rights', avg(rating)=5.0),
 Row(tag='free to download', avg(rating)=5.0),
 Row(tag='thought provoking', avg(rating)=4.75),
 Row(tag='parrots', avg(rating)=4.75),
 Row(tag='jon hamm', avg(rating)=4.75),
 Row(tag='Dystopia', avg(rating)=4.75),
 Row(tag='movies about movies', avg(rating)=4.666666666666667)]